#Licence Validation

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  # draw face bounding box on image
#   for (x,y,w,h) in faces:
#       img = cv2.rectangle(img,(x-50,y-50),(x+w+70,y+h+50),(255,0,0),2)
#   # save image
  cv2.imwrite(filename, img)

  return filename

In [ ]:
try:
  filename = take_photo('snapshot.jpg')
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
# !pip install gtts

In [ ]:
# !pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 15.7 MB/s eta 0:00:00


In [ ]:
import easyocr
import cv2

In [ ]:
lang_list=["te","en"]

In [ ]:
reader = easyocr.Reader(lang_list)
result = reader.readtext('licence.jpg')
result

In [ ]:
from datetime import datetime
from datetime import date
def difference(exp):
    print(exp)
    # dates in string format
    str_d1 = exp
    str_d2 = date.today().strftime("%d/%m/%Y")  # Convert datetime.date to string
    print(str_d1[0:2], str_d2[0:2])
    print(str_d1[3:5], str_d2[3:5])
    print(str_d1[-4:], str_d2[-4:])
    print(str_d1, str_d2)
    if int(str_d1[-4:]) - int(str_d2[-4:]) > 0:
        delta = int(str_d1[-4:]) - int(str_d2[-4:])  # Note: changed to d1 - d2 to calculate days until expiry
        return delta
    if int(str_d1[-4:]) == int(str_d2[-4:]):
        if int(str_d1[3:5]) >  int(str_d2[3:5]):
            delta = int(str_d1[3:5]) - int(str_d2[3:5])  # Note: changed to d1 - d2 to calculate days until expiry
            return delta
    if int(str_d1[-4:]) == int(str_d2[-4:]):
        if int(str_d1[3:5]) ==  int(str_d2[3:5]):
            if int(str_d1[0:2]) >=  int(str_d2[0:2]):
                delta = int(str_d1[-4:]) - int(str_d2[-4:])  # Note: changed to d1 - d2 to calculate days until expiry
                return delta
    return -1

In [ ]:
import pandas as pd

df = pd.read_csv('License_Data.csv')
df

In [ ]:
# k = dates_sep()
    # exp = valid(k)
    # exp = str(exp)
    # print(exp)
    # out = difference(str(exp))
    # print(out)

In [ ]:
import re

flag = 0
temp = 0
for coord in result:
    k = str(coord[1]).lower()  # Convert to string
    x = re.search('lic+', k)
    y = re.search('driv', k)
    if x:
        flag = 1
        break
    if y:
        flag = 1
        break

print(flag)

if flag == 1:
    column_name = 'LIC_NUM'
    for coord in result:
        string = str(coord[1])  # Convert to string
        print(string)
        if len(string) == 17 or len(string) == 16:
            filtered_df = df[column_name][df[column_name].str.contains(string)]
            values = filtered_df.item()  # Retrieve the single value as string
            print(values)
            if not filtered_df.empty:  # Check if DataFrame is not empty
                search_idx = filtered_df.index
                print('-----------------------')
                y = (df['EXPIRY_DATE'][search_idx]).item()
                print(y)
                print('-----------------------')
                valid = difference(y)
                print(valid)
                if valid >= 0:
                    print("Valid Licence")
                    message = 'Valid Licence, Place the key to start your vehicle'
                    temp = 1
                    break

if flag == 0 or temp == 0:
    print("Please scan proper licence")
    message = "Please scan proper licence"


In [ ]:
from gtts import gTTS
from IPython.display import Audio
tts = gTTS(message)
tts.save('1.wav')
sound_file = '1.wav'
Audio(sound_file, autoplay=True)

#Face Comparision

Currently working on it.

In [ ]:
!pip install dlib

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install --upgrade dlib


In [ ]:
import cv2
import dlib

# Load the picture image
picture_image = cv2.imread('me.jpg')  # Replace 'picture.jpg' with the actual path to the picture image

# Create a face detector and load the pre-trained face recognition model
detector = dlib.get_frontal_face_detector()
face_recognizer = dlib.face_recognition_model_v1('/content/dlib_face_recognition_resnet_model_v1.dat')
  # Path to the pre-trained face recognition model

# Compute the face encodings for the picture image
picture_face = detector(picture_image)[0]
picture_face_encoding = face_recognizer.compute_face_descriptor(picture_image, picture_face)

# Create a VideoCapture object for live streaming (e.g., webcam)
video_capture = cv2.VideoCapture(0)  # Use 0 for the default webcam, or replace it with the appropriate index if you have multiple cameras

# Read frames from the live stream until the comparison is made or user quits
while True:
    # Read frame from the live stream
    ret, frame = video_capture.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    # Iterate through detected faces
    for face in faces:
        # Compute the face encodings for the current face in the frame
        face_encoding = face_recognizer.compute_face_descriptor(frame, face)

        # Compare the current face encoding with the encoding from the picture
        match = dlib.distance(face_encoding, picture_face_encoding) < 0.6

        # Determine if the faces belong to the same person
        if match:
            comparison_result = "Same Person"
        else:
            comparison_result = "Different Person"

        # Draw a rectangle around the face and display the comparison result
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, comparison_result, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Face Comparison', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close windows
video_capture.release()
cv2.destroyAllWindows()
